<a href="https://colab.research.google.com/github/KellieChong/E2Metrix-Membrane-Fouling-Model/blob/main/E2metrix_Membrane_Fouling_Mechanism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we will import the necessary packages.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import math
#import os
#import io
#from six.moves import urllib
from google.colab import files
from scipy.optimize import minimize_scalar, fmin
from mpl_toolkits import mplot3d

In [ ]:
# Now we will add the csv files from my github
url001 = "https://raw.githubusercontent.com/KellieChong/E2Metrix-Membrane-Fouling-Model/79d29886800a484d0e519d409848c40f4225547b/200%20nm%20-%2020%20psi/01-20-2021-001.csv"
url002 = "https://raw.githubusercontent.com/KellieChong/E2Metrix-Membrane-Fouling-Model/79d29886800a484d0e519d409848c40f4225547b/200%20nm%20-%2020%20psi/01-20-2021-002.csv"
url003 = "https://raw.githubusercontent.com/KellieChong/E2Metrix-Membrane-Fouling-Model/79d29886800a484d0e519d409848c40f4225547b/200%20nm%20-%2020%20psi/01-20-2021-003.csv"
url004 = "https://raw.githubusercontent.com/KellieChong/E2Metrix-Membrane-Fouling-Model/79d29886800a484d0e519d409848c40f4225547b/200%20nm%20-%2020%20psi/01-20-2021-004.csv"

# Data frames are now to be loaded
df001 = pd.read_csv(url001)
df002 = pd.read_csv(url002)
df003 = pd.read_csv(url003)
df004 = pd.read_csv(url004)

# Plot our spliced raw data files for visualization
'''plt.plot(df001["time"], df001["J_v"], label = "0-1h")
plt.plot(df002["time"], df002["J_v"], label = "1-2h")
plt.plot(df003["time"], df003["J_v"], label = "2-3h")
plt.plot(df004["time"], df004["J_v"], label = "3-4h")
plt.title = ("Normalized Flux vs. Time")
plt.xlabel = ("time (h)")
plt.ylabel = ("Normalized Flux (LMH)")
plt.legend()
plt.show()'''

ax = df001.plot(x="time", y = "J_v", label = "0-1h", xlabel = "time (h)", ylabel = "Normalized Flux (LMH)", title = "Normalized Flux Vs. Time", color = "hotpink")
df002.plot(x = "time", y = "J_v", label = "1-2h", color = "mediumslateblue", ax=ax)
df003.plot(x = "time", y = "J_v", label = "2-3h", color = "midnightblue", ax=ax)
df003.plot(x = "time", y = "J_v", label = "3-4h", color = "lightseagreen", ax=ax)

In [ ]:
# Now we'll create the functions and store them into arrays to later be compared to our raw data to see the mechanism of fouling
#4 possibilities exist: 1:Complete Pore Blocking, 2: Intermediate Blocking, 3: Pore constriction, and 4: Cake filtration
database = [[df001["J_v"][0], df002["J_v"][0], df003["J_v"][0], df004["J_v"][0]], #the Jo values
            [df001["J_v"], df002["J_v"], df003["J_v"], df004["J_v"]], # J_v values
            [df001["time"], df002["time"], df003["time"], df004["time"]]] # time intervals

# Let's the user choose the test of interest
test = widgets.Dropdown(options = [("0-1h", 0), ("1-2h", 1), ("2-3h", 2), ("3-4h", 3)], description = "Choose your dataset of interest:")

# These are all column vectors
Y = database[1][test.value].to_numpy()/database[0][test.value]
t = database[2][test.value]
K_block = np.linspace(0, 1.0, len(t))

x, y = np.meshgrid(t, K_block)
z = np.square(Y - np.exp(-x*y))
#def K_block (K_block, t):
#    sol1 = np.exp((np.transpose(K_block)) * -t)
#    return sol1

#print(Y)
#t_reshaped = np.reshape(-t, (1, len(t)))
#K_b_reshaped = np.reshape(K_block, (len(t), 1))
#y_pred = np.reshape(K_block, (len(t), 1)) * np.reshape(-t, (1, len(t)))
Y_mat = np.tile(np.array(np.reshape(Y, (1, len(t)))), (len(t), 1))
#print(Y_mat.shape)
y_pred = np.zeros((len(t), len(t)))

i=0
j=0

for i in range(len(t)):
  for j in range(len(t)):
    y_pred[i][j] = np.exp(K_block[i] * -t[j])
    if j == len(t):
      break
    else:
      j += 1
  i += 1
  if i == len(t):
    break

RSS = np.square((Y_mat - y_pred))

#print(RSS)
#print(y_pred)

#def RSS(K_block, t, Y):
    #y_pred = K_block(K_block, t)
 #   residual_sum = sum((Y - y_pred) ** 2)
 #   return residual_sum

#print(np.reshape(K_block, (len(t),1)))
#fig1 = plt.figure(figsize =(6,6))
#ax = fig1.add_subplot(projection='3d')
#ax.plot_surface(x, y, z)
#plt.xlabel("time (h)")
#plt.ylabel("K_block")
#ax.set_zlabel("RSS")

#plt.imshow(RSS)
#plt.colorbar()
plt.show()
#res = minimize_scalar(RSS, bounds=(-100, 100), method= 'bounded')
#res.x 

display(test)


In [ ]:
Y = database[1][test.value].to_numpy()/database[0][test.value]
t = database[2][test.value]
K_block = np.linspace(0, 1.0, len(t))

x, y = np.meshgrid(t, K_block)
z1 = np.square(Y - np.exp(-x*y))

zmin = np.min(z1)
mask = np.array(1) == zmin
color = np.where(mask,'red', 'blue')

fig1 = plt.figure(figsize =(6,6))
ax = fig1.add_subplot(111, projection='3d')
ax.plot_surface(x, y, z1)
plt.xlabel("time (h)")
plt.ylabel("K_block")
ax.set_zlabel("RSS")
plt.show()
print(z1 in z1==zmin)


In [ ]:
from scipy.optimize import minimize

ALPHA_TRUE = 0.5 # used only to generate some test data

def model(t, K_block):
    # here you need to implement your real model
    # for Predicted_Installation
    y_pred = np.exp(K_block * -t)
    return y_pred

def sum_of_squares(Y):
    y_pred = model(t, K_block)
    obj = np.sqrt(((y_pred - Y) ** 2).sum())
    return obj


# generate some test data
X = time
Y = model([ALPHA_TRUE], X) # Original_Installation


# perform fit to find optimal parameters
# initial value for alpha (guess)
alpha_0 = 0.1

res = minimize(sum_of_squares, [alpha_0, ], args=(X, Y), tol=1e-3, method="Powell")
print(res)